# ***TASK 3***

Description:

"Write a function that takes as input two timestamps of the form 2017/05/13 12:00 and calculates their differences in hours. Please only return the full hour difference and round the results. E.g. 2022/02/15 00:05 and 2022/02/15 01:00 would return 1 hour."

In [1]:
import pandas as pd
from typing import List

In [2]:
def calculate_hours(t1: str, t2: str) -> int:
    """
    Calculates difference in hours between `t1` and `t2` in hours, rounded to the nearest full hour.
    :param t1: str, YYYY/mm/dd HH:MM formatted timestamp. Start datetime.
    :param t2: str, YYYY/mm/dd HH:MM formatted timestamp. End datetime.
    :return: int
    """
    return round((pd.Timestamp(t2) - pd.Timestamp(t1)).total_seconds() / 3600)

# ***TASK 4***

Description:

"Expand the above function to only count the time difference between 09:00 – 17:00 and only on weekdays."

In [3]:
def calculate_business_hours(t1: str, t2: str, 
                             bday_start: int = 9, bday_end: int = 17,
                             workdays: List = [0, 1, 2, 3, 4]) -> int:
    """
    Calculates the difference in hours between `t1` and `t2` in business hours, where business rules
        applied at input. Returns number of hours as integer rounded to nearest whole hour.
    :param t1: str, YYYY/mm/dd HH:MM formatted timestamp. Start datetime.
    :param t2: str, YYYY/mm/dd HH:MM formatted timestamp. End datetime.
    :param bday_start: int, business day start in 24 hour time, e.g. 9am -> 9
    :param bday_end: int, business day end in 24 hour time, e.g. 5am -> 17
    :param workdays: list, zero-based days of week to consider as workdays, e.g. 0 -> Monday, 6 -> Sunday
    :return: int
    """
    start = pd.Timestamp(t1).floor('H')  # floor hour of start
    start_sec = (pd.Timestamp(t1).ceil('H') - pd.Timestamp(t1)).seconds  # seconds of first hour to count
    
    end = pd.Timestamp(t2).ceil('H')  # ceil hour of end
    end_sec = (pd.Timestamp(t2) - pd.Timestamp(t2).floor('H')).seconds # seconds of last hour to count
    
    ind = pd.date_range(start=start, end=end, freq='H')  # create arr of hours between start & end
    working_hours = [i for i in range(bday_start, bday_end+1)]
    
    df = pd.concat([pd.Series(ind).dt.dayofweek.rename('dow'),
                pd.Series(ind).dt.hour.rename('hr')], axis=1).set_index(ind)
    
    # Total seconds in a full hour
    df['sec'] = 3600
    
    # Replace seconds in start & end hour
    df.iloc[0, -1] = start_sec
    df.iloc[-1, -1] = end_sec
    
    # Remove extra hour, else double count last/first
    return round((df[(df['dow'].isin(workdays)) & 
                     (df['hr'].isin(working_hours))]['sec'].sum() - 3600) / 3600)
    